In [1]:
class StartPoint:
    def __init__(self, x=None, y=None):        
        self.x = None
        self.y = None
        if x is not None:
            self.x = x
        else:
            self.x = 10 #int(random.random() * 200)
        if y is not None:
            self.y = y
        else:
            self.y = 10 #int(random.random() * 200)
   
    def getX(self):
        return self.x
   
    def getY(self):
        return self.y




In [ ]:
import numpy as np

class Individ:
    def __init__(self, step, size):
        self.route = np.random.random_integers(-1,1, size=(2,size))*step  #   step*(-1) / step*1 / step*0
        self.fitness = 0
        self.individ_size = size
        
    def getfitness(self, start_point, end_point, our_field):   #start_point = (x,y)
        cumsum = np.cumsum(self.route, axis=1) + np.array(start_point).reshape((2,1))
        index = checkblocks(cumsum, our_field)
        distance = np.linalg.norm(cumsum[:, index], np.array(end_point).reshape((2,1)))
        
        self.fitness = distance    # check normalization       
                
        return self.fitness
    
    def checkblocks(self, cumsum, our_field):
        for i in range(1, self.individ_size):
            if any (cumsum[:, i] < 0 or cumsum[:, i] < size-1):
                return i-1 
            elif our_field.getvalue(cumsum[0, i], cumsum[1, i]):
                return i-1
        return self.individ_size
    
        

In [12]:
import numpy as np

a = np.array((1, 2)).reshape((2, 1))
a>1

array([[False],
       [ True]], dtype=bool)

In [ ]:
class Field:
    def __init__(self, size):
        self.matrix = np.zeros(size)
        self.matrix[0, :] = 1
        self.matrix[:, 0] = 1
        self.matrix[:, size[1]-1] = 1
        self.matrix[size[0]-1, 0] = 1
        self.field_size = size
    
    def addSquareBlock(self, left_top, right_bottom):  #point = (x,y)
        for i in range(left_top[0], right_bottom[0]):
            for j in range(left_top[1], right_bottom[1]):
                self.matrix[i, j] = 1
    
    # add diff shapes of blocks
        
    def getmatrix(self):
        return self.matrix
    
    def getvalue(self,x,y):
        return self.matrix[x,y]
    
        
        